<a href="https://colab.research.google.com/github/noamZigler/Sentiment_Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install praw pandas nltk textblob transformers torch  requests beautifulsoup4 langdetect
import praw
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from google.colab import files
import requests
from bs4 import BeautifulSoup
from langdetect import detect,DetectorFactory
nltk.download("vader_lexicon")
nltk.download('punkt_tab')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [40]:
def determine_sentiment(sentiment_score):
    if sentiment_score >= 0.05:
        return "Positive"
    elif sentiment_score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

def pre_procces_AAPL_Reddit(search_results,posts_data):
  for post in search_results:
    post_text = post.selftext if post.selftext else post.title  # Use post body if available, else title
    sentiment_score = sia.polarity_scores(post_text)["compound"]  # VADER sentiment analysis
    sentiment = determine_sentiment(sentiment_score)
    posts_data.append({
          "title": post.title,
          "num_comments": post.num_comments,
          "upVotes ratio": post.upvote_ratio,
          "sentiment_score": sentiment_score,
          "sentiment": sentiment
      })
  return posts_data

In [41]:
#Tasks 2-4
reddit = praw.Reddit(
    client_id='6yGwpzY2BBE4aoiNwwO1mg',
    client_secret='rHUvbGCJXRgNU0ROW4SHUV02p0N7PQ',
    user_agent='AAPL sentiment analysis  -Noam Zigler'
)
subreddit = reddit.subreddit("all")
search_results = subreddit.search("AAPL", limit=100)
posts_data=[]
# Convert to DataFrame
df = pd.DataFrame(pre_procces_AAPL_Reddit(search_results,posts_data))
df


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



,title,num_comments,upVotes ratio,sentiment_score,sentiment
0,PUTS ON $AAPL,373,0.91,0.7779,Positive
1,AAPL Retire Us,110,0.87,0.9391,Positive
2,RIP AAPL (short term),54,0.73,0.0516,Positive
3,Are you worried about AAPL long term?,462,0.78,-0.9311,Negative
4,Anyone else trim their AAPL position?,66,0.62,0.9001,Positive
...,...,...,...,...,...
95,Recently inherited ~1100 shares in AAPL. Shoul...,358,0.91,0.9876,Positive
96,Morgan Stanley boosts Apple (AAPL) Stock Targe...,133,0.96,0.5106,Positive
97,AAPL has grown their market cap by $800 billio...,185,0.96,0.5410,Positive
98,What's the take on AAPL's future given Trump v...,244,0.65,0.2265,Positive


In order to take in to account the virality of a post we first need to define what is virality. For us the characteristics that we took in account are:

*   Number of comments on the post
*   UpVote ratio
*   score- number of up votes



Now we conduct the following formula for the virality factor:
$$
\text{Virality Factor} = 1+ \left(( \log(1 + \text{Score}) \right) \times \left( 1 + \frac{\text{Comments}}{100} \right) \times \text{Upvote Ratio})
$$
 In the end the new sentiment score will be : $$
\text{New sentiment score} =\text{ sentiment score} * \text{Virality Factor}
$$

In [42]:
def virality_factor(post):
  return 1 + ((torch.log(1 + post["score"])) * (1 + post["num_comments"] / 100) * post["upVotes ratio"])
def new_sentiment_score(post):
  return post["sentiment_score"] * virality_factor(post)

**Functions for part 7**

In [43]:

def get_full_article_content(article_url):
    try:
        page = requests.get(article_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        paragraphs = soup.find_all('p')
        full_text = ' '.join([para.get_text() for para in paragraphs])
        return full_text if full_text else None
    except:
        return None
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [45]:

API_KEY = '8e45e7989a50476d91832f7fe665b64e'
url = 'https://newsapi.org/v2/everything'
params = {
    'q': ['AAPL','apple'],
    'language': 'en',
    'pageSize': 100,
    'apiKey': API_KEY
}
response = requests.get(url, params=params)
data = response.json()
articles = data.get('articles', [])
#filter empty articles ,articles that are not in english and articles that needs cookies
filtered_articles = [
    article for article in articles
    if article.get('content')
    and is_english(article.get('content'))
    and not article.get('content').startswith("If you click 'Accept all'")
]
for article in filtered_articles:
    full_content = get_full_article_content(article.get('url', ''))
    if full_content:
        article['content'] = full_content

df = pd.DataFrame(filtered_articles)
df = df[['title', 'description','content']]
df['sentiment_score'] = df['content'].apply(lambda text: sia.polarity_scores(text)['compound'])
df['sentiment']= df['sentiment_score'].apply(determine_sentiment)
df.sample(n=5)

,title,description,content,sentiment_score,sentiment
61,"Apple Stock Drops Amid Soft iPhone Sales, Zuck...",Apple stock continued its downward trajectory ...,\n Access to this page has been...,0.4374,Positive
7,Apple earnings are coming. Here's what to expect,Apple (AAPL) is expected to release its fiscal...,Apple (AAPL+1.74%) is expected to release its ...,0.9741,Positive
62,TikTok's Supreme Court date is tomorrow. Here'...,The nation’s highest court is preparing to hea...,The nation’s highest court is preparing to hea...,0.9974,Positive
14,The Nasdaq soars 350 points as Nvidia stock re...,The Nasdaq and other major indexes recovered o...,The Nasdaq and other major indexes rebounded o...,0.8689,Positive
24,Why Is Cliff Asness Bullish On Apple Inc. (AAPL)?,We recently compiled a list of the 10 Best Sto...,We are experiencing some temporary issues. The...,0.9983,Positive
